# Import dependencies and libraries

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import networkx
from textblob import TextBlob

import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

# Twitter API authentication

In [2]:
# keys and tokens from the Twitter Dev Console 
consumer_key = '4Skk3kFGkHb0ryTGLvRNcrN3M'
consumer_secret = 'O37XRtd5HsPy05r2H6eJnx2nWWIcVrbnVJXWDwPmPcCacmrZMx'
access_token = '1214729489730678784-vfBALf5kgI1YsORCj8fiktzaqPwUKW'
access_token_secret = '3BoSyzrQW6UibJNU493N64rF5BZRdEIFjatAqcS2yNicG'

# Twitter API authentication
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

# Custom search criteria

In [8]:
# Create a custom search term and define the number of tweets
ticker="TSLA"
search_term = "#ticker -filter:retweets"

# Functions to return a dataframe -  a list of cleaned up tweets for custom search criteria , its score and polarity 

As we agreed to just pull information based on user input for 1 ticker at a time, functions are created below to pull for a particular ticker

In [4]:
def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """

    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

def tweets():
    tweets = tw.Cursor(api.search,
                   q=search_term,
                   lang="en",
                   since="2020-01-07").items(10000)


    tweet_data={}
    for tweet in tweets:
        tweet_date = tweet._json['created_at']
        tweets_no_urls = remove_url(tweet.text)
        tweet_data.update({tweet_date:tweets_no_urls}) #{tweet_date:tweets_no_urls}
    print (tweet_data)


    # Remove URLs
    tweets_no_urls = [remove_url(tweet.text) for tweet in tweets]

    # Store dict keys and values in separate lists
    values=tweet_data.values()
    dates=tweet_data.keys()

    # Create textblob objects of the tweets
    sentiment_objects = [TextBlob(tweet) for tweet in values]
    # sentiment_objects[0].polarity, sentiment_objects[0]

    # Create list of polarity valuesx and tweet text
    sentiment_values = [[str(tweet),dates,ticker,tweet.sentiment.polarity,'Positive' if tweet.sentiment.polarity > 0 else 'Negative' if tweet.sentiment.polarity < 0 else 'Neutral'] for tweet in sentiment_objects]
    # sentiment_values[0]

    # Create dataframe containing the polarity value and tweet text
    sentiment_df = pd.DataFrame(sentiment_values, columns=[ "Tweet","date","Ticker","Score","Polarity"])
    sentiment_df["date"]=dates
    return sentiment_df
    
    
    

In [5]:
AAPL_tweets=tweets()
AAPL_tweets

{'Wed Jan 15 03:12:26 +0000 2020': 'I think it should really be dealt case by case AAPL Apple which I think they already do when provided with a cour', 'Wed Jan 15 02:47:48 +0000 2020': 'Dont believe me Read the testimonials We are growing small accounts daily Join the Ne', 'Wed Jan 15 02:43:56 +0000 2020': 'ESF W 115RedGreen possibleDataEarningsTradeNews33073302Poss HOD32973292R328832863283Pivot32', 'Wed Jan 15 02:27:18 +0000 2020': 'SPY Support and Resistance Pivots for Wed 11520FB AMZN AAPLGOOG TSLA CBAC BMY IBBMSFT NFLX', 'Wed Jan 15 00:42:28 +0000 2020': 'ES SPY NASDAQ AAPL Bubble bubble toil and trouble 6 Month chart', 'Wed Jan 15 00:39:42 +0000 2020': 'MadMoneyOnCNBCPresident Trump is mad at AAPLIs that a reason for the pullbackCancelling doctors appointment o', 'Wed Jan 15 00:39:17 +0000 2020': 'Mr President is time to Sell AAPL more than 100 already realDonaldTrump OK Tweet Sent OptionsMike', 'Wed Jan 15 00:11:03 +0000 2020': 'petenajarian Mr Najarian you mentioned on Halftime

,Tweet,date,Ticker,Score,Polarity
0,I think it should really be dealt case by case...,Wed Jan 15 03:12:26 +0000 2020,AAPL,0.200000,Positive
1,Dont believe me Read the testimonials We are g...,Wed Jan 15 02:47:48 +0000 2020,AAPL,-0.125000,Negative
2,ESF W 115RedGreen possibleDataEarningsTradeNew...,Wed Jan 15 02:43:56 +0000 2020,AAPL,0.000000,Neutral
3,SPY Support and Resistance Pivots for Wed 1152...,Wed Jan 15 02:27:18 +0000 2020,AAPL,0.000000,Neutral
4,ES SPY NASDAQ AAPL Bubble bubble toil and trou...,Wed Jan 15 00:42:28 +0000 2020,AAPL,-0.200000,Negative
5,MadMoneyOnCNBCPresident Trump is mad at AAPLIs...,Wed Jan 15 00:39:42 +0000 2020,AAPL,-0.625000,Negative
6,Mr President is time to Sell AAPL more than 10...,Wed Jan 15 00:39:17 +0000 2020,AAPL,0.500000,Positive
7,petenajarian Mr Najarian you mentioned on Half...,Wed Jan 15 00:11:03 +0000 2020,AAPL,0.000000,Neutral
8,The end of the AAPL parabolic run,Tue Jan 14 23:54:27 +0000 2020,AAPL,0.000000,Neutral
9,esf3280 could be used as support for on sessio...,Tue Jan 14 23:39:31 +0000 2020,AAPL,0.000000,Neutral


In [7]:
TSLA_tweets=tweets()
TSLA_tweets

{'Wed Jan 15 03:01:05 +0000 2020': 'Could Be Bigger Than Netflix adThese 5 stocks could be bigger than Netflix or Amazon Join now and start investi', 'Wed Jan 15 00:03:39 +0000 2020': '1 Stock for the 5G Tech Revolution ad5G is 100x faster than outdated 4G One American smart chip is FULL D', 'Tue Jan 14 21:00:29 +0000 2020': 'Protect your IRA401k adUsing Gold amp Silver Receive a Free Precious Metals Investment Guide DOWNLOAD HERE', 'Tue Jan 14 18:02:34 +0000 2020': 'The Top 5G Stock in America adBlackRock JP Morgan Goldman Sachs and Citi have taken HUGE positions in this 5', 'Tue Jan 14 15:02:17 +0000 2020': 'Are your Stocks on the Rise adPull a FREE stock report that tells you if a stock is buy or sell No guesswork', 'Tue Jan 14 04:34:19 +0000 2020': 'detail Moneyscape pietradura Marble Inlay from the ongoing series Triumph Of Market 2012market', 'Tue Jan 14 03:00:53 +0000 2020': 'Trading Secrets Revealed adVeteran trader Bryan Bottarelli has revealed all of his tricks to win every d

,Tweet,date,Ticker,Score,Polarity
0,Could Be Bigger Than Netflix adThese 5 stocks ...,Wed Jan 15 03:01:05 +0000 2020,TSLA,0.000000,Neutral
1,1 Stock for the 5G Tech Revolution ad5G is 100...,Wed Jan 15 00:03:39 +0000 2020,TSLA,0.041071,Positive
2,Protect your IRA401k adUsing Gold amp Silver R...,Tue Jan 14 21:00:29 +0000 2020,TSLA,0.450000,Positive
3,The Top 5G Stock in America adBlackRock JP Mor...,Tue Jan 14 18:02:34 +0000 2020,TSLA,0.450000,Positive
4,Are your Stocks on the Rise adPull a FREE stoc...,Tue Jan 14 15:02:17 +0000 2020,TSLA,0.400000,Positive
5,detail Moneyscape pietradura Marble Inlay from...,Tue Jan 14 04:34:19 +0000 2020,TSLA,0.000000,Neutral
6,Trading Secrets Revealed adVeteran trader Brya...,Tue Jan 14 03:00:53 +0000 2020,TSLA,0.800000,Positive
7,BREAKTHROUGH adA small undertheradar biotech f...,Tue Jan 14 00:04:30 +0000 2020,TSLA,0.016667,Positive
8,Eight Seconds 1260 Richer adThis mans inventio...,Mon Jan 13 21:03:01 +0000 2020,TSLA,0.000000,Neutral
9,1 EyePopping Stock To Buy Now adThis tiny comp...,Mon Jan 13 18:04:01 +0000 2020,TSLA,0.045455,Positive
